In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Get list of behaviors

In [3]:
def get_behaviors_list(exp_dir):
    """
    Finds first JAABA experiment and returns list of all behavioral files.
    """
    flybowl_path = os.path.join(exp_dir, next(os.walk(exp_dir))[1][0])
    return list(filter(lambda x: x.startswith('scores'), os.listdir(flybowl_path)))
behaviors_list = get_behaviors_list(os.path.join('/content/drive/MyDrive/0 Flybowl/2025 FlyBowl JABBA data/2-5-2025-30207JAABAData'))
print(behaviors_list)

['scores_Walk.mat', 'scoresTouch.mat', 'scores_Stops.mat', 'scores_pivot_center.mat', 'scores_Jump.mat', 'scores_Righting.mat', 'scores_Crabwalk3.mat', 'scores_Crabwalk2.mat', 'scores_Copulation.mat', 'scores_Chasev7.mat', 'scores_Chaining.mat', 'scores_BodyTurns.mat', 'scores_pivot_tail.mat', 'scoresBackup.mat', 'scores_AttemptedCopulation.mat']


# Set up folder structure for data storage

In [ ]:
folder_name = '/content/drive/MyDrive/0 Flybowl/2025 FlyBowl JABBA data/2-5-2025-30207JAABAData'
for behavior in behaviors_list:
    os.mkdir(os.path.join('/content/drive/MyDrive/0 Flybowl/line/102864 data', behavior[behavior.find("_")+1:behavior.rfind(".")]))

# Get experimental groupings for each line

In [5]:
def get_exp_groupings(exp_dir, subfolders):
  groupings_list = []
  for subfolder in subfolders:
    for exp in os.listdir(os.path.join(exp_dir, subfolder)):
      exp_path = os.path.join(exp_dir, subfolder, exp)
      grouping = exp.rsplit(' ')[-1][1:2]
      groupings_list.append([exp_path, grouping])
  return pd.DataFrame(groupings_list, columns=['path', 'group'])

dir_name = '/content/drive/MyDrive/0 Flybowl/2025 FlyBowl JABBA data'
folders_30207 = ['2-5-2025-30207JAABAData', '2-12-2025-30207JAABAData']
paths_30207 = get_exp_groupings(dir_name, folders_30207)

folders_102864 = ['2-20-2025-102864JAABAData', '3-4-2025-102864JAABAData', '3-26-2025-102864JAABAData']
paths_102864 = get_exp_groupings(dir_name, folders_102864)

# Extract and Bar Plot Data

In [11]:
def get_data(flybowls, beh):
  max_length = 0
  all_data = None
  for flybowl in flybowls:
    t0 = sp.io.loadmat(os.path.join(flybowl, beh))['allScores'][0][0][5][0]
    t1 = sp.io.loadmat(os.path.join(flybowl, beh))['allScores'][0][0][6][0]

    n_flies = t0.shape[0]
    flybowl_data = np.zeros((n_flies, 19800))
    for fly in range(n_flies):
      t0_i, t1_i = t0[fly][0], t1[fly][0]
      for i in range(len(t0_i)):
        flybowl_data[fly][t0_i[i]:t1_i[i]] = 1

    flybowl_data = np.sum(flybowl_data, axis=0)
    if all_data is None:
      all_data = flybowl_data
    else:
      all_data = np.vstack([all_data, flybowl_data])
  return all_data

def clean_data(uncleaned_data):
  df = pd.DataFrame(np.transpose(uncleaned_data))

  bins = np.arange(0, 66)*30*10
  df['bins'] = pd.cut(df.index, bins, right=False)
  df = df.groupby('bins').sum()[0:60]

  return df

def graph(data_exp, data_c1, data_c2,  beh):
  exp_mean = np.mean(data_exp, axis=0)
  exp_sem = sp.stats.sem(data_exp)

  c1_mean = np.mean(data_c1, axis=0)
  c1_sem = sp.stats.sem(data_c1)

  c2_mean = np.mean(data_c2, axis=0)
  c2_sem = sp.stats.sem(data_c2)

  fig, ax = plt.subplots()
  fig.set_size_inches(20, 10)

  ax.plot(exp_mean, color='deepskyblue', alpha=0.666, label='Experimental')
  ax.fill_between(np.arange(0, 60), exp_mean-exp_sem, exp_mean+exp_sem, color='deepskyblue', alpha=.2)

  ax.plot(c1_mean, color='lightgray', alpha=0.666, label='Control 1')
  ax.fill_between(np.arange(0, 60), c1_mean-c1_sem, c1_mean+c1_sem, color='lightgray', alpha=.2)

  ax.plot(c2_mean, color='darkgray', alpha=0.666, label='Control 2')
  ax.fill_between(np.arange(0, 60), c2_mean-c2_sem, c2_mean+c2_sem, color='darkgray', alpha=.2)

  leg = ax.legend(fontsize=12, frameon=False)
  for legobj in leg.get_lines():
      legobj.set_linewidth(3)

  ax.set_title(beh, fontsize=16)
  ax.set_xlabel('Time (min)', fontsize=14)
  ax.set_ylabel('Occurrences', fontsize=14)

  ax.set_xticks(np.arange(0, 11)*6)
  ax.set_xticklabels(np.arange(11), fontsize=12)
  ax.tick_params(axis='y', labelsize=12)

  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)

  plt.savefig(f'/content/drive/MyDrive/0 Flybowl/line/30207 plots/{beh}.pdf')

In [ ]:
current_exp = paths_30207
for beh in behaviors_list:
  exp_data = clean_data(get_data(current_exp[current_exp['group'] == '1']['path'], beh))
  c1_data = clean_data(get_data(current_exp[current_exp['group'] == '2']['path'], beh))
  c2_data = clean_data(get_data(current_exp[current_exp['group'] == '3']['path'], beh))

  beh_name = beh[beh.find("_")+1:beh.rfind(".")]
  exp_data.to_csv(f'/content/drive/MyDrive/0 Flybowl/line/30207 data/{beh_name}/exp.csv')
  c1_data.to_csv(f'/content/drive/MyDrive/0 Flybowl/line/30207 data/{beh_name}/c1.csv')
  c2_data.to_csv(f'/content/drive/MyDrive/0 Flybowl/line/30207 data/{beh_name}/c2.csv')
  graph(np.array(exp_data).T, np.array(c1_data).T, np.array(c2_data).T, beh_name)